In [1]:
#!pip install transformers
#!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.4 MB/s 
     |████████████████████████████████| 6.6 MB 57.8 MB/s 
     |████████████████████████████████| 596 kB 67.5 MB/s 
     |████████████████████████████████| 101 kB 12.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [4]:
import pandas as pd
import numpy as np

import torch
import gc

gc.collect()
torch.cuda.empty_cache()

In [15]:
df_raw = pd.read_json('kinopoisk.jsonl',
                      encoding='utf-8',
                      lines=True)
df_raw.sample(5)

,part,movie_name,review_id,author,date,title,grade3,grade10,content
33934,bottom100,Волкодав (2006),25235,Mishell,2007-10-25,жаль,Neutral,0.0,"\n Этот фильм мог бы стать шедевром, если..."
27619,top250,Терминатор 2 Судный день (1991),12105,zombion,2010-11-22,Как я вспомнил Арни! 5 часть.,Good,10.0,\nМне нужна твоя одежда…\n\nТерминатор 2: Судн...
11615,top250,Изгой (2000),14420,stee1y,2010-02-09,Если друг оказался вдруг..,Good,8.0,"\nЭто интересно — поймать себя на мысли, что в..."
34934,bottom100,Самый лучший фильм 3-ДЭ (2011),25848,A_i_s,2011-03-06,"-«Кто такой Ленин?» — «Ээ… Тот оборотень, лови...",Bad,3.0,\n Так теперь будет выглядеть ответ школь...
12398,top250,"Весна, лето, осень, зима... и снова весна (2003)",24386,Joseph Mon,2006-12-28,None,Good,0.0,\nФильм невероятный по силе духа и смыслу. В н...


In [35]:
df_norm = df_raw.loc[:, ('content',
                         'grade3')]
df_norm.rename(columns={'content': 'text',
                        'grade3': 'label'},
               inplace=True)
df_norm.label = df_norm.label.map({'Good': 2,
                                   'Neutral': 1,
                                   'Bad': 0})
df_norm.sample(10)

,text,label
5172,\n«Запах женщины» — фильм заслуживающий приста...,2
15039,"\nДа, давно я смотрел этот фильм, еще в школьн...",2
13214,"\nИстория о животном, отдавшем больше, чем оно...",2
2676,\nМногие пытались снять фильм про воров. Я пос...,2
25296,\nСамый худший фильм в карьере Скосезе и можно...,0
11843,\nСтивен Спилберг умеет снимать только масштаб...,2
27999,\nПосмотрел я этот фильм только сейчас. И не р...,1
26346,\nЭто действительно так. Фильм — фееричен. Это...,2
21154,\n«С ума сойти можно!» — это была первая мысль...,2
9403,"\nСамобытная, новаторская картина. В полной ме...",2


In [60]:
from sklearn.model_selection import train_test_split

# Тестовая и тренировочная части
train_df, test_df = train_test_split(df_norm, 
                                     test_size = 0.3,
                                     random_state = 1234)

In [61]:
from datasets import DatasetDict, Dataset

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
    "valid": Dataset.from_pandas(test_df.reset_index(drop=True)),})

In [62]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25613
    })
    valid: Dataset({
        features: ['text', 'label'],
        num_rows: 10978
    })
})

In [63]:
dataset['train'][0]

{'label': 2,
 'text': '\nВ Декларации независимости США\xa0записано, что\xa0все люди имеют право на\xa0стремление к\xa0счастью. Почему не\xa0право на\xa0счастье? Да\xa0потому что\xa0неотъемлемое право на\xa0счастье\xa0—\xa0абсурд. И\xa0дело не\xa0в том, что\xa0по каким-то абстрактным максимам счастье нужно заслужить. Просто счастье лишь на\xa0десятую часть (а то\xa0и меньше) определяется достигнутым результатом. Все\xa0остальное\xa0—\xa0осознание тех\xa0усилий, которые мы\xa0приложили, того пути, который прошли, прежде чем\xa0оказаться там, где\xa0мы есть. И\xa0того, что\xa0все было не\xa0зря. Если бы\xa0главному герою дали высокооплачиваемую работу лишь за\xa0то, что\xa0он смог сложить кубик Рубика, был\xa0бы он\xa0счастлив? Да, конечно. Но\xa0испытал бы\xa0он те\xa0же самые эмоции, почувствовал бы\xa0такое же счастье? Сомневаюсь.\n\nКрис Гарднер\xa0—\xa0обычный парень, у\xa0которого есть жена и\xa0маленький сын, съемная квартира и\xa0кабальная работа. Они\xa0едва сводят концы с\xa0ко

In [64]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny2')

loading file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/16af2afaa4ceaa8d50b689bd4c2f7ef7fe3bfac06c0aac7d82a5c1c72298b62a.cc3312d07ccf88871a3c2b7cb3442138e6785101efead94d9f77e96301cf7f4a
loading file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/cfa6d82dc8ecc7fe3f06deb449f38968cdd188bb84c8da0e06f0bbfddbede1e3.550ab7157d36210bf96c7c3b30e621933d37d635c5f2e290f7e88bd5f7c9198a
loading file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/20317640533199c6b37a557395cd5ee5fcb8777be7c89bb1314bfd43058b35e9.dd8bd9bfd3664b530ea4e645105f557769387b3da9f79bdb55ed556bdd80611d
loading file https://huggingface.co/cointegrated/rubert-tiny2/resolv

In [65]:
def preprocess_function(examples):
    return tokenizer(examples['text'], 
                     padding='max_length',
                     truncation=True,
                     max_length=512)

In [66]:
tokenized_dataset = dataset.map(preprocess_function,
                                batched = True)

  0%|          | 0/26 [00:00<?, ?ba/s]

  0%|          | 0/11 [00:00<?, ?ba/s]

In [67]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [68]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained('cointegrated/rubert-tiny2', 
                                                           num_labels=3)

loading configuration file https://huggingface.co/cointegrated/rubert-tiny2/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/726139048d10597682731a7a4c0b8ef0382927911bc0f6f050a4f7f0afb04c2a.149cdc07694f3925e290abc5528c57a543bcbc9af955c0202b1028584ad15cb4
Model config BertConfig {
  "_name_or_path": "cointegrated/rubert-tiny2",
  "architectures": [
    "BertForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "emb_size": 312,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 600,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "pad_token_id": 0,
  "position_embeddin

In [69]:
from datasets import load_metric

metric = load_metric('accuracy')

def compute_metrics(eval_pred):
  logits, labels = eval_pred
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions,
                        references=labels)

In [70]:
batch_size = 32

In [71]:
from transformers import TrainingArguments, Trainer

In [72]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    save_strategy='no',
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=4,
    weight_decay=0.01,
    evaluation_strategy='epoch'
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [73]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['valid'],
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [74]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 25613
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 3204


Epoch,Training Loss,Validation Loss,Accuracy
1,0.613000,0.418419,0.841957
2,0.395800,0.395427,0.848971
3,0.372600,0.392287,0.849608
4,0.346700,0.393309,0.849882


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10978
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10978
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 10978
  Batch size = 32
The follow

TrainOutput(global_step=3204, training_loss=0.4145547745379616, metrics={'train_runtime': 957.2854, 'train_samples_per_second': 107.023, 'train_steps_per_second': 3.347, 'total_flos': 755600764391424.0, 'train_loss': 0.4145547745379616, 'epoch': 4.0})

In [78]:
df_metrics = pd.read_csv('performance_table.csv',
                          encoding='utf-8')
df_metrics = df_metrics.append({'Model':'Rubert',
                                'Test Accuracy': '84.99',
                                'Precision': '0.85',
                                'Recall': '0.85',
                                'F1': '0.85'},
                               ignore_index=True)
df_metrics

,Model,Test Accuracy,Precision,Recall,F1
0,Multinomial Naive Bayes,62.25,0.62,0.62,0.62
1,Multinomial Naive Bayes+,62.25,0.62,0.62,0.62
2,SVC,61.55,0.62,0.62,0.62
3,SVC+,66.95,0.67,0.67,0.67
4,Logistic Regression,66.76,0.67,0.67,0.67
5,Logistic Regression+,66.76,0.67,0.67,0.67
6,Random Forest,63.97,0.64,0.64,0.64
7,Random Forest+,63.97,0.64,0.64,0.64
8,Gradient Boosting,62.06,0.62,0.62,0.62
9,Gradient Boosting+,62.08,0.62,0.62,0.62


In [53]:
trainer.save_model('models/rubert2')

Saving model checkpoint to models/rubert2
Configuration saved in models/rubert2/config.json
Model weights saved in models/rubert2/pytorch_model.bin
tokenizer config file saved in models/rubert2/tokenizer_config.json
Special tokens file saved in models/rubert2/special_tokens_map.json


In [55]:
from transformers import pipeline

inferer = pipeline('text-classification', 
            model='models/rubert2', 
            tokenizer=trainer.tokenizer,
            function_to_apply="sigmoid", 
            return_all_scores=True,
            device=0)

loading configuration file models/rubert2/config.json
Model config BertConfig {
  "_name_or_path": "models/rubert2",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "emb_size": 312,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 312,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 600,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 3,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.20.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 83828
}

loading configuration fil

In [58]:
example = '''
Признаться, сериал был до недавнего времени одним из лучший тайлов эвер. Первый сезон, вышедший осенью 2016-го своей атмосферой тайны, заговора и дней минувший заставлял приклеиться к экранам компа всех. Стильно и оригинально было всё, а дети, обычно являющиеся чёрной дырой в сериалах там затаскивали похлеще взрослых. Но как известно хорошие сериалы помирают в муках и агонии. Очень не хотелось такого конца для Странных дел, однако 3 сезон с 'ненавязчивым' заделом на 4 стал превращать сериал во всё большую пародию на самого себя как случилось с Сидячими болтунами.<br />
<br />
Что не так:<br />
<br />
1) Потраченная атмосфера. Тут пожалуй без комментариев, оболочка осталась, а вот начинка исчезла.<br />
<br />
2) Любовные линии. Как бы не уверяли меня создатели, что детки-то подросли и пора обращать на противоположный пол внимание, я в этой истории Уилл Байерс.<br />
<br />
3) Испорченное поведение. Основная шестёрка ведёт себя по канону, а вот на шерифе Джиме Хоппере Дафферы отыгрались. Что это за истеричка в пубертатном периоде, переживающая не о деле, а о том, что его кинули/могут кинуть/Джойс может с кем-то встречаться помимо него и т.д. и т.п.<br />
<br />
4) Мотивация и главзлодеи. О да, извечная тема плохих рюсских. Настолько заигрались в переодевания (в прямом смысле), что мотивацию этим плохим рюсским открывать врата в жопу демона я так и не услышала, поправьте меня, если я глухая.<br />
<br />
Плюсом, как-то даже обидно, что сверхсекретная 'наша' база с полпинка находится подростками с самопальной рацией.<br />
<br />
5) Отсюда выливается пятый пункт о полнейшей неправдоподобности вообще всего происходящего.<br />
<br />
Итог: посмотреть продолжение ради посмотреть продолжение, 4 сезон особо не жду.</span></p>
 '''

In [59]:
inferer('example')

[{'label': 'LABEL_0', 'score': 0.4593772888183594},
 {'label': 'LABEL_1', 'score': 0.6036779284477234},
 {'label': 'LABEL_2', 'score': 0.43949198722839355}]

**Выводы:**


Классические модели машинного обучения достаточно плохо показали себя в задаче многоклассовой классификации отзывов по их тональности.

Классические модели с трудом отделяли нейтральные отзывы от положительных и отрицательных.

Наиболее эффективным решением оказалось использование трансвормера RuBert Tiny 2 со встроенной прдобработкой текстов.

Можно исследовать и другие русскоязычные трансформеры.